# Day 6: Session A - Data Manipulation and Analysis Techniques

[Link to session webpage](https://eds-217-essential-python.github.io/course-materials/interactive-sessions/6a_grouping_joining_sorting.html)

Sorting, grouping, joining, applying


## Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# Create a sample dataset of species observations
np.random.seed(42) # seed is initial condition
# don't use a seed unless you have a good reason to (i.e. teaching)
dates = pd.date_range(start='2023-01-01', periods=100)
data = { # dictionary
    'date': dates,
    'site': np.random.choice(['Forest', 'Grassland', 'Wetland'], 100),
    'species': np.random.choice(['Oak', 'Maple', 'Pine', 'Birch'], 100),
    'count': np.random.randint(1, 20, 100),
    'temperature': np.random.normal(15, 5, 100)
}
df = pd.DataFrame(data)
print(df.head())

        date     site species  count  temperature
0 2023-01-01  Wetland   Birch      7     9.043483
1 2023-01-02   Forest   Birch      1    18.282768
2 2023-01-03  Wetland   Birch      1    10.126592
3 2023-01-04  Wetland    Pine     13    18.935423
4 2023-01-05   Forest    Pine      9    20.792978


## 1. Sorting Data

### Basic Sorting

In [25]:
# Sort by species count
df_sorted = df.sort_values('count', ascending=False)
print(df_sorted.head())

         date       site species  count  temperature
81 2023-03-23     Forest   Birch     19    19.262167
12 2023-01-13     Forest     Oak     19    10.552428
53 2023-02-23     Forest   Maple     19    20.677828
55 2023-02-25    Wetland   Birch     19    18.256956
33 2023-02-03  Grassland   Maple     19    19.281994


### Multi-column Sorting

We can change sort order between diff rows

In [26]:
# Sort by site and then by species count
df_multi_sorted = df.sort_values(['site', 'count'], ascending=[True, False])
print(df_multi_sorted.head())
# reads from left to rightSort all the forests first, by highest count forests
# then grasslands by highest count grasslands, etc...
# don't need to use copy! It auto makes a new df. not a view

         date    site species  count  temperature
12 2023-01-13  Forest     Oak     19    10.552428
53 2023-02-23  Forest   Maple     19    20.677828
81 2023-03-23  Forest   Birch     19    19.262167
61 2023-03-03  Forest     Oak     18    15.409371
95 2023-04-06  Forest   Maple     18    20.162326


This will break your data frame! You took values out, sorted, and pasted it back in

```python
 df['count'] = df['count'].sort_values(ascending=False) 
```

## 2. Grouping and Aggregating Data

### Basic group by

In [27]:
# Sum of species counts by site
site_counts = df.groupby('site')['count'].sum()
#                group by,       select,  aggregate
print(site_counts)

site
Forest       311
Grassland    336
Wetland      248
Name: count, dtype: int64


### Multiple aggregations

Provide a list of aggregations instead of just one function using `agg()`

Using agg and groupby is super powerful!!

In [28]:
# For one column, let's get the result of multiple aggrations:
# pass the commands in as strings, which is a little weird
site_stats_count = df.groupby('site')['count'].agg(['sum', 'mean', 'max'])
# need to use functions as strings because without, you would use the system's min which is much slower


In [29]:
# Provide column specific aggregations in a dictionary
# Multiple stats by site
site_stats = df.groupby('site').agg({
    'count': ['sum', 'mean', 'max'],
    'species': 'nunique',
    'temperature': 'mean'
})
print(site_stats)

          count               species temperature
            sum      mean max nunique        mean
site                                             
Forest      311  9.424242  19       4   16.527332
Grassland   336  9.333333  19       4   15.540037
Wetland     248  8.000000  19       4   14.528127


## Joining Data

Arguments
1. initial df (positional, have to be first)
2. new df (positional, have to be second)
3. on='site' <- column you want to join on
4. how='inner' <- joining method

In [30]:
# Create a second DataFrame with site characteristics
site_data = pd.DataFrame({
    'site': ['Forest', 'Grassland', 'Wetland'],
    'soil_pH': [6.5, 7.2, 6.8],
    'annual_rainfall': [1200, 800, 1500]
})

In [31]:
# Perform an inner join
# 
merged_df = pd.merge(df, site_data, on='site', how='inner')
print(merged_df.head())

        date     site species  count  temperature  soil_pH  annual_rainfall
0 2023-01-01  Wetland   Birch      7     9.043483      6.8             1500
1 2023-01-02   Forest   Birch      1    18.282768      6.5             1200
2 2023-01-03  Wetland   Birch      1    10.126592      6.8             1500
3 2023-01-04  Wetland    Pine     13    18.935423      6.8             1500
4 2023-01-05   Forest    Pine      9    20.792978      6.5             1200


## 4. Working with Dates

For time series data, it's nice to make the row index a timestamp

Set index to change the index

In [33]:
new_df = merged_df.set_index('date')
new_df.index.month

Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4],
      dtype='int32', name='date')

### Understanding `inplace=True`

If you are calling a method that usually generates a new df (like `set_index()`, `sort_values()`) and you want the operation to act on the dataframe without making a copy...

Then you can use the `inplace` keyword argument to force this behavior

There is a tendency to bloat data space. Can avoid this by
1. Editing in place
2. Chaining operations

In [45]:
df_copy = merged_df.copy() #insurance
print("without in place \n", df_copy.head())

# without inplace keyword argument, we get new df
df_new = df_copy.set_index('date')
print(df_new.head())

df_copy.set_index('date', inplace=True)
print("\nwith in place \n", df_copy.head())


without in place 
         date     site species  count  temperature  soil_pH  annual_rainfall
0 2023-01-01  Wetland   Birch      7     9.043483      6.8             1500
1 2023-01-02   Forest   Birch      1    18.282768      6.5             1200
2 2023-01-03  Wetland   Birch      1    10.126592      6.8             1500
3 2023-01-04  Wetland    Pine     13    18.935423      6.8             1500
4 2023-01-05   Forest    Pine      9    20.792978      6.5             1200
               site species  count  temperature  soil_pH  annual_rainfall
date                                                                     
2023-01-01  Wetland   Birch      7     9.043483      6.8             1500
2023-01-02   Forest   Birch      1    18.282768      6.5             1200
2023-01-03  Wetland   Birch      1    10.126592      6.8             1500
2023-01-04  Wetland    Pine     13    18.935423      6.8             1500
2023-01-05   Forest    Pine      9    20.792978      6.5             1200

with i